In [82]:
import urllib, time, json, csv
import numpy as np
import pandas as pd

In [86]:
write_path = 'auctions.json'
# for finding api
def finding_data(page):
    api_key = 'CharlesZ-11ce-4517-8df3-becc92494006'

    paramKeys = ['OPERATION-NAME', 'SERVICE-VERSION','SECURITY-APPNAME','RESPONSE-DATA-FORMAT','categoryId']
    paramVals = ['findCompletedItems','1.7.0',api_key,'JSON&REST-PAYLOAD','9355']


    url_finding = 'http://svcs.ebay.com/services/search/FindingService/v1?'
    url = url_finding
    for i in range(len(paramKeys)):
        url += paramKeys[i] + '=' + paramVals[i] + '&'

    response = urllib.urlopen(url + 'page=' + str(page))
    data = json.loads(response.read())
    
    # parse data for selected auctions
    return data['findCompletedItemsResponse'][0]['searchResult'][0]['item']



In [90]:
# finding api
# loop backwards to avoid conflicts
with open(write_path,"w") as outfile:
    for i in range(0,200):
        data = finding_data(200-i)
        time.sleep(1) 
        json.dump(data,outfile)
        print "page", 200-i, "complete"
    

page 200 complete
page 199 complete
page 198 complete
page 197 complete
page 196 complete
page 195 complete
page 194 complete
page 193 complete
page 192 complete
page 191 complete
page 190 complete
page 189 complete
page 188 complete
page 187 complete
page 186 complete
page 185 complete
page 184 complete
page 183 complete
page 182 complete
page 181 complete
page 180 complete
page 179 complete
page 178 complete
page 177 complete
page 176 complete
page 175 complete
page 174 complete
page 173 complete
page 172 complete
page 171 complete
page 170 complete
page 169 complete
page 168 complete
page 167 complete
page 166 complete
page 165 complete
page 164 complete
page 163 complete
page 162 complete
page 161 complete
page 160 complete
page 159 complete
page 158 complete
page 157 complete
page 156 complete
page 155 complete
page 154 complete
page 153 complete
page 152 complete
page 151 complete
page 150 complete
page 149 complete
page 148 complete
page 147 complete
page 146 complete
page 145 c

In [103]:
with open(write_path,'r') as fd:
    reader = json.loads(fd)
    
    

TypeError: expected string or buffer